In [ ]:
import importlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import seaborn as sns
from sklearn import metrics
import torch

from models import ff, persistent, rnn
from utils import data_utils, data_loader, model_utils, shape_utils

importlib.reload(data_utils)
importlib.reload(data_loader)
importlib.reload(shape_utils)

%matplotlib inline

RUN_FOLDER = "../results/debug/"
NETWORK = "kcm/"
FOLD_MODEL = 4
device = torch.device("cpu")
NUM_WORKERS = 0
HIDDEN_SIZE = 32
BATCH_SIZE = 512

In [ ]:
kcm_inputs = data_utils.load_all_inputs(RUN_FOLDER, "kcm/")
atb_inputs = data_utils.load_all_inputs(RUN_FOLDER, "atb/")

# Make dataloaders
kcm_dataloader = data_loader.make_generic_dataloader(
    kcm_inputs['test_data'],
    kcm_inputs['config'],
    BATCH_SIZE,
    data_loader.basic_collate,
    NUM_WORKERS
)
atb_dataloader = data_loader.make_generic_dataloader(
    atb_inputs['test_data'],
    atb_inputs['config'],
    BATCH_SIZE,
    data_loader.basic_collate,
    NUM_WORKERS
)
kcm_dataloader_seq = data_loader.make_generic_dataloader(
    kcm_inputs['test_data'],
    kcm_inputs['config'],
    BATCH_SIZE,
    data_loader.sequential_collate,
    NUM_WORKERS
)
atb_dataloader_seq = data_loader.make_generic_dataloader(
    atb_inputs['test_data'],
    atb_inputs['config'],
    BATCH_SIZE,
    data_loader.sequential_collate,
    NUM_WORKERS
)
_, kcm_mask_seq = data_utils.get_seq_info(kcm_dataloader_seq)
_, atb_mask_seq = data_utils.get_seq_info(atb_dataloader_seq)

# Define embedded variables for nn models
kcm_embed_dict = {
    'timeID': {
        'vocab_size': 1440,
        'embed_dims': 24
    },
    'weekID': {
        'vocab_size': 7,
        'embed_dims': 4
    }
}
# Define embedded variables for nn models
atb_embed_dict = {
    'timeID': {
        'vocab_size': 1440,
        'embed_dims': 24
    },
    'weekID': {
        'vocab_size': 7,
        'embed_dims': 4
    }
}

In [ ]:
# Load trained models
kcm_model = ff.FF(
    "FF",
    11,
    HIDDEN_SIZE,
    kcm_embed_dict,
    device
)
kcm_model.load_state_dict(torch.load(f"{RUN_FOLDER}kcm/models/FF_{FOLD_MODEL}.pt"))
kcm_model.eval()
kcm_model_seq = rnn.GRU_RNN(
    "GRU_RNN",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    kcm_embed_dict,
    device
)
kcm_model_seq.load_state_dict(torch.load(f"{RUN_FOLDER}kcm/models/GRU_RNN_{FOLD_MODEL}.pt"))
kcm_model_seq.eval()

atb_model = ff.FF(
    "FF",
    11,
    HIDDEN_SIZE,
    atb_embed_dict,
    device
)
atb_model.load_state_dict(torch.load(f"{RUN_FOLDER}atb/models/FF_{FOLD_MODEL}.pt"))
atb_model.eval()
atb_model_seq = rnn.GRU_RNN(
    "GRU_RNN",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    atb_embed_dict,
    device
)
atb_model_seq.load_state_dict(torch.load(f"{RUN_FOLDER}atb/models/GRU_RNN_{FOLD_MODEL}.pt"))
atb_model_seq.eval()

In [ ]:
# Load baseline models
kcm_model_baseline = data_utils.load_pkl(f"{RUN_FOLDER}kcm/models/AVG_{FOLD_MODEL}.pkl")
kcm_model_baseline_seq = data_utils.load_pkl(f"{RUN_FOLDER}kcm/models/PER_TIM_{FOLD_MODEL}.pkl")
atb_model_baseline = data_utils.load_pkl(f"{RUN_FOLDER}atb/models/AVG_{FOLD_MODEL}.pkl")
atb_model_baseline_seq = data_utils.load_pkl(f"{RUN_FOLDER}atb/models/PER_TIM_{FOLD_MODEL}.pkl")
all_results = []

In [ ]:
# Test KCM on KCM
train_network = "KCM"
test_network = "KCM"
labels, preds = kcm_model_baseline.predict(kcm_dataloader)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{kcm_model_baseline.model_name}"
}))
labels, preds, avg_loss = model_utils.predict(kcm_model, kcm_dataloader)
labels = data_utils.de_normalize(labels, kcm_inputs['config']['time_mean'], kcm_inputs['config']['time_std'])
preds = data_utils.de_normalize(preds, kcm_inputs['config']['time_mean'], kcm_inputs['config']['time_std'])
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{kcm_model.model_name}"
}))
labels, preds = kcm_model_baseline_seq.predict(kcm_dataloader_seq)
preds = data_utils.aggregate_tts(preds, kcm_mask_seq)
labels = data_utils.aggregate_tts(labels, kcm_mask_seq)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{kcm_model_baseline_seq.model_name}"
}))
labels, preds, avg_loss = model_utils.predict(kcm_model_seq, kcm_dataloader_seq, sequential_flag=True)
labels = data_utils.de_normalize(labels, kcm_inputs['config']['time_gap_mean'], kcm_inputs['config']['time_gap_std'])
preds = data_utils.de_normalize(preds, kcm_inputs['config']['time_gap_mean'], kcm_inputs['config']['time_gap_std'])
preds = data_utils.aggregate_tts(preds, kcm_mask_seq)
labels = data_utils.aggregate_tts(labels, kcm_mask_seq)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{kcm_model_seq.model_name}"
}))

In [ ]:
# Test KCM on ATB
train_network = "KCM"
test_network = "ATB"
labels, preds = kcm_model_baseline.predict(atb_dataloader)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{kcm_model_baseline.model_name}"
}))
labels, preds, avg_loss = model_utils.predict(kcm_model, atb_dataloader)
labels = data_utils.de_normalize(labels, kcm_inputs['config']['time_mean'], kcm_inputs['config']['time_std'])
preds = data_utils.de_normalize(preds, kcm_inputs['config']['time_mean'], kcm_inputs['config']['time_std'])
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{kcm_model.model_name}"
}))
labels, preds = kcm_model_baseline_seq.predict(atb_dataloader_seq)
preds = data_utils.aggregate_tts(preds, atb_mask_seq)
labels = data_utils.aggregate_tts(labels, atb_mask_seq)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{kcm_model_baseline_seq.model_name}"
}))
labels, preds, avg_loss = model_utils.predict(atb_model_seq, atb_dataloader_seq, sequential_flag=True)
labels = data_utils.de_normalize(labels, kcm_inputs['config']['time_gap_mean'], kcm_inputs['config']['time_gap_std'])
preds = data_utils.de_normalize(preds, kcm_inputs['config']['time_gap_mean'], kcm_inputs['config']['time_gap_std'])
preds = data_utils.aggregate_tts(preds, atb_mask_seq)
labels = data_utils.aggregate_tts(labels, atb_mask_seq)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{kcm_model_seq.model_name}"
}))

In [ ]:
# Test ATB on ATB
train_network = "ATB"
test_network = "ATB"
labels, preds = atb_model_baseline.predict(atb_dataloader)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{atb_model_baseline.model_name}"
}))
labels, preds, avg_loss = model_utils.predict(atb_model, atb_dataloader)
labels = data_utils.de_normalize(labels, atb_inputs['config']['time_mean'], atb_inputs['config']['time_std'])
preds = data_utils.de_normalize(preds, atb_inputs['config']['time_mean'], atb_inputs['config']['time_std'])
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{atb_model.model_name}"
}))
labels, preds = atb_model_baseline_seq.predict(atb_dataloader_seq)
preds = data_utils.aggregate_tts(preds, atb_mask_seq)
labels = data_utils.aggregate_tts(labels, atb_mask_seq)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{atb_model_baseline_seq.model_name}"
}))
labels, preds, avg_loss = model_utils.predict(atb_model_seq, atb_dataloader_seq, sequential_flag=True)
labels = data_utils.de_normalize(labels, atb_inputs['config']['time_gap_mean'], atb_inputs['config']['time_gap_std'])
preds = data_utils.de_normalize(preds, atb_inputs['config']['time_gap_mean'], atb_inputs['config']['time_gap_std'])
preds = data_utils.aggregate_tts(preds, atb_mask_seq)
labels = data_utils.aggregate_tts(labels, atb_mask_seq)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{atb_model_seq.model_name}"
}))

In [ ]:
# Test ATB on KCM
train_network = "ATB"
test_network = "KCM"
labels, preds = atb_model_baseline.predict(kcm_dataloader)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{atb_model_baseline.model_name}"
}))
labels, preds, avg_loss = model_utils.predict(atb_model, kcm_dataloader)
labels = data_utils.de_normalize(labels, atb_inputs['config']['time_mean'], atb_inputs['config']['time_std'])
preds = data_utils.de_normalize(preds, atb_inputs['config']['time_mean'], atb_inputs['config']['time_std'])
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{atb_model.model_name}"
}))
labels, preds = atb_model_baseline_seq.predict(kcm_dataloader_seq)
preds = data_utils.aggregate_tts(preds, kcm_mask_seq)
labels = data_utils.aggregate_tts(labels, kcm_mask_seq)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{atb_model_baseline_seq.model_name}"
}))
labels, preds, avg_loss = model_utils.predict(atb_model_seq, kcm_dataloader_seq, sequential_flag=True)
labels = data_utils.de_normalize(labels, atb_inputs['config']['time_gap_mean'], atb_inputs['config']['time_gap_std'])
preds = data_utils.de_normalize(preds, atb_inputs['config']['time_gap_mean'], atb_inputs['config']['time_gap_std'])
preds = data_utils.aggregate_tts(preds, kcm_mask_seq)
labels = data_utils.aggregate_tts(labels, kcm_mask_seq)
all_results.append(pd.DataFrame({
    "Labels": labels,
    "Preds": preds,
    "Train Network": train_network,
    "Test Network": test_network,
    "Model": f"{atb_model_seq.model_name}"
}))

In [ ]:
gen_df = pd.concat(all_results)
gen_df['Residuals'] = abs(gen_df['Labels'] - gen_df['Preds'])
gen_df

In [ ]:
metric_results = gen_df.groupby(['Model','Train Network','Test Network'], as_index=False).apply(lambda x: np.round(np.sqrt(metrics.mean_squared_error(x.Labels, x.Preds)), 2))
metric_results.columns = ["Model","Train Network","Test Network","Metric"]
metric_results['Train-Test-Model'] = metric_results['Train Network']+"_"+metric_results['Test Network']+"_"+metric_results['Model']
metric_results = metric_results.sort_values('Train-Test-Model')
metric_results

In [ ]:
fig, axes = plt.subplots(1,1)
fig.set_figheight(6)
fig.set_figwidth(6)
sns.barplot(metric_results, x="Metric", y="Train-Test-Model")
axes.set_ylabel("Model")
axes.set_xlabel("RMSE")
fig.suptitle('Model Generalization Performance', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_generalization.png", dpi=1800, bbox_inches='tight')

In [ ]:
plot_data = gen_df[gen_df['Labels']<10000]
g = sns.FacetGrid(data=plot_data, hue="Model", col="Test Network", row="Train Network", height=4)
g.map(sns.scatterplot, "Labels", "Residuals")
plt.savefig(f"../plots/model_generalization_preds.png", dpi=1800, bbox_inches='tight')